In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm


In [2]:
# Load the data
df = pd.read_csv('height.dat', sep='\t')

# Display the first few rows to check the data
print(df.head())

print(df.columns)

df.info()


   Part  Operator  Repetition    Value
0     1         1           1  488.418
1     2         1           1  484.798
2     3         1           1  489.445
3     4         1           1  485.758
4     5         1           1  489.043
Index(['Part', 'Operator', 'Repetition', 'Value'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Part        40 non-null     int64  
 1   Operator    40 non-null     int64  
 2   Repetition  40 non-null     int64  
 3   Value       40 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 1.4 KB


In [3]:
# Fit the ANOVA model
model = ols('Value ~ C(Part) + C(Operator) + C(Part):C(Operator)', data=df).fit()

# Perform the ANOVA
anova_results = anova_lm(model)

print(anova_results)



                       df      sum_sq    mean_sq           F        PR(>F)
C(Part)               9.0  276.387954  30.709773  117.041911  2.599400e-15
C(Operator)           1.0    5.511578   5.511578   21.005873  1.802444e-04
C(Part):C(Operator)   9.0    3.058264   0.339807    1.295082  2.992650e-01
Residual             20.0    5.247654   0.262383         NaN           NaN


In [4]:
ms_part = anova_results['mean_sq']['C(Part)']
ms_operator = anova_results['mean_sq']['C(Operator)']
ms_repeatability = anova_results['mean_sq']['Residual']

# Number of repetitions per part per operator
n_repetitions = df['Repetition'].nunique()



In [5]:
# Variance components
repeatability_variance = ms_repeatability
reproducibility_variance = (ms_operator - ms_repeatability) / n_repetitions
part_to_part_variance = (ms_part - ms_repeatability) / (n_repetitions * df['Operator'].nunique())

# Total GR&R Variance (sum of repeatability and reproducibility)
total_grr_variance = repeatability_variance + reproducibility_variance

# Total variance (sum of part-to-part and total GR&R variance)
total_variance = total_grr_variance + part_to_part_variance


In [6]:
# Given tolerance range
tolerance_range = 495 - 480  # This is 15 units

# Standard deviation for Total GR&R
std_total_grr = np.sqrt(total_grr_variance)

# Percent Tolerance for Total GR&R
percent_tolerance_grr = (std_total_grr / tolerance_range) * 100
print(f"Percent Tolerance for Total GR&R: {percent_tolerance_grr:.2f}%")


Percent Tolerance for Total GR&R: 11.33%


In [7]:
# Percent Contribution for Total GR&R
percent_contribution_grr = (total_grr_variance / total_variance) * 100
print(f"Percent Contribution for Total GR&R: {percent_contribution_grr:.2f}%")


Percent Contribution for Total GR&R: 27.50%
